In [30]:
library("DESeq2")
library("ggplot2")
library("ggrepel")
library("ggcorrplot")
library("dplyr")
library(stringr)
library(purrr)
library("tibble")
library(dplyr)
library(tidyr)
library(ComplexHeatmap)
library("pals")
library(ggpubr)
library(tximport)
library("ggExtra")
source('./plot_data.R')
theme_set(
    theme_classic(base_size = 12)
)

## Figure1B

In [24]:
tpm<- read.table('../data/1_total_rna_rsem_iso_TPM.tsv')
tpm_ave<- cbind(data.frame(WT = rowSums(tpm[, c(1:3)])/3), data.frame(MUT = rowSums(tpm[, c(4:6)])/3))
final_iso_tb<- read.table('../data/2_final_iso_tb.tsv', header = T, row.names = 1, sep = '\t')
tpm_ave$class = final_iso_tb[match(rownames(tpm_ave), rownames(final_iso_tb)), 'class']
tpm_ave<- tpm_ave %>% filter(!is.na(class))
tpm_ave$class<- factor(tpm_ave$class, levels = c('low expression', 'medium expression', 'high expression'))

In [25]:
tpm_ave$class %>% table()

.
   low expression medium expression   high expression 
             1476              3538              1278 

In [51]:
mcol <- rev(c("#ca0020", "#f4a582", "#92c5de"))
names(mcol) <- levels(tpm_ave$class)
intercept = 0
slope = 1

reg_fun <- function(x) intercept + slope * x

p<- tpm_ave %>% 
    ggplot(aes(x  = log2(WT+1), y = log2(MUT+1))) +
    geom_point(aes(color = class), size = 0.5) +
    stat_function(fun = reg_fun, color = "black", linetype = "dashed", linewidth = 1, alpha = 0.8) +
    scale_color_manual(values = mcol) +
    labs(color = "Expression") +
    theme(legend.position = "bottom", legend.background = element_blank(), 
          legend.title = element_text(hjust = 40, vjust = 2, size = 12)) +
    guides(color = guide_legend(override.aes = list(size = 4)))

p2<- ggMarginal(p, groupColour = TRUE, groupFill = TRUE, type = "densigram", binwidth = 0.2, size =4)

ggsave('../figures/Figure1B_total_rna_gene_expression_class.pdf', p2, width = 4, height = 4)

Warning message in ggplot2::geom_density(colour = "NA", ...):
“Ignoring unknown parameters: `binwidth`”
Warning message in (function (mapping = NULL, data = NULL, stat = "identity", position = "identity", :
“Ignoring unknown parameters: `binwidth`”
Warning message in ggplot2::geom_density(colour = "NA", ...):
“Ignoring unknown parameters: `binwidth`”
Warning message in (function (mapping = NULL, data = NULL, stat = "identity", position = "identity", :
“Ignoring unknown parameters: `binwidth`”
